In [55]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
from imblearn.over_sampling import SMOTE

In [56]:
df = pd.read_csv(r"C:\MasterThesis\v1.0\sub-004\ses-001\dataset.csv")
labels = pd.read_csv(r"C:\MasterThesis\v1.0\sub-004\ses-001\ieeg\sub-004_ses-001_task-FR_run-01_events_encoding.tsv", sep='\t')
labels = labels['is_success']
X = df
y = labels

In [57]:
sm = SMOTE(random_state=0)
X, y = sm.fit_resample(X, y)

In [58]:
# normalizacja danych
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [60]:
model = SVC(kernel='linear')

# trenowanie modelu
model.fit(X_train, y_train)

# przewidywanie na danych testowych
predictions = model.predict(X_test)

# sprawdzanie dokładności modelu
accuracy = accuracy_score(y_test, predictions)

print(f"Accuracy: {accuracy*100}%")

Accuracy: 57.14285714285714%
